In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import numpy as np
import pandas as pd
import string

In [4]:
import requests
from bs4 import BeautifulSoup
import re

def extract_email_from_company_website(url):
    try:
        # Set a browser user-agent so the website doesn't block the request as a bot
        headers = {"User-Agent": "Mozilla/5.0"}

        # Send an HTTP GET request to the company's website
        res = requests.get(url, headers=headers, timeout=10)

        # If the request failed (e.g., 404 or timeout), return "N/A"
        if res.status_code != 200:
            return "N/A"

        # Parse the website content (HTML) using BeautifulSoup
        soup = BeautifulSoup(res.text, "html.parser")

        # Get all the text from the page (without tags) to search for emails
        text = soup.get_text()

        # Use a regular expression to find email patterns in the text
        emails = re.findall(
            r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+", text
        )

        # Return the first email found, or "N/A" if no email was found
        return emails[0] if emails else "N/A"

    except Exception as e:
        # If any error occurs (network issue, bad HTML, etc.), return "N/A"
        print(f"Error scraping {url}: {e}")
        return "N/A"


# Collect Data

In [ ]:
# Setup driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
companies_collected = 0
# Open website
driver.get("https://www.hotfrog.co.id")

# Wait and search
search_bar = driver.find_element(By.XPATH, "/html/body/header/div[2]/div/div[2]/form/div/div[1]/input")
search_bar.send_keys("Motorcycle Spare Parts")

search_click = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "/html/body/header/div[2]/div/div[2]/form/div/button"))
)
try:
    search_click.click()
except Exception as e:
    print(e)

WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "serps")))
#print("Entered the Motorcycle Page")

start_company = 1
end_company = 18
page = 1


name = []
email = []
phone = []
address = []
while page < 115:
    start_company = 1
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "serps")))

    companies = driver.find_elements(By.CSS_SELECTOR, ".hf-box.clearfix")
    print(f"Found {len(companies)} companies on page {page}")

    for i in range(len(companies)):
        try:
            company_header = companies[i].find_element(By.CLASS_NAME, "h6.mb-0")
            company_link = company_header.find_element(By.TAG_NAME, 'a')

            driver.execute_script("arguments[0].scrollIntoView(true);", company_link)
            href = company_link.get_attribute("href")
            #the_link = href.replace("company", "message")
            if not href or "javascript:void" in href:
                start_company+=1
                continue
            link = company_link.get_attribute("href")
            #print(f"Entered {link}")
            # Scroll into view
            time.sleep(1)

            # Try clicking
            try:
                company_link.click()
            except:
                # Fallback to JS click
                driver.execute_script("arguments[0].click();", company_link)

            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "bdp")))

            #print("Entered Company's Page")


            # Name
            Name = driver.find_element(By.CSS_SELECTOR, ".lead.hfhl").text
            name.append(Name)
            print(f"Company {start_company} : {Name}, Page {page}")
            company_url = "N/A"
            try:
                company_url_elem = driver.find_element(By.XPATH, "//i[contains(@class, 'fa-link')]/following-sibling::a[1]")
                company_url = company_url_elem.get_attribute("href")
                print("Company website:", company_url)
            except:
                print("No external website found.")

            ## Here add the logic of finding comanies url
            # Email-Phone-Address
            contact = driver.find_element(By.ID, "bdp-contacts")
            temp = contact.find_elements(By.TAG_NAME, "dt")
            phone_done = True
            email_done = True
            address_done = True

        
            for j in range(len(temp)):
                label = temp[j].text.strip().lower()
                #print(label)

                if label in ["mobile", "telepon", "phone"] and phone_done == True:
                    try:
                        dd = temp[j].find_element(By.XPATH, "following-sibling::dd[1]")
                        phone.append(dd.text)
                        #print(dd.text)
                        phone_done = False
                    except:
                        phone.append(np.nan) 

                elif label in ["alamat", "address"]:
                    try:
                        dd = temp[j].find_element(By.XPATH, "following-sibling::dd[1]")
                        
                        address.append(dd.text)
                        address_done = False
                        #print(dd.text)
                    except:
                        address.append(np.nan)
            # If something is missed
            if email_done == True:
                if company_url == "N/A":
                    email.append(np.nan)
                else:
                    fallback_email = extract_email_from_company_website(company_url) if company_url != "N/A" else "N/A"
                    email.append(fallback_email)

            if phone_done == True:
                phone.append(np.nan)
            if address_done == True:
                address.append(np.nan)
        
        #
            driver.back()
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "serps")))
            #print("Left Company's Page")
            companies = driver.find_elements(By.CSS_SELECTOR, ".hf-box.clearfix")
            companies_collected+=1
        except Exception as e:
            print(f"Error at company {start_company}: {e}")
        start_company += 1
        
    print(f"Page : {page} completed")
    try:
        pagination_container = driver.find_element(By.CSS_SELECTOR, ".py-3.bg-white")
        next_page_link = pagination_container.find_elements(By.CSS_SELECTOR, ".page-link.border-0")
        next_page_link = next_page_link[-1]
        print(next_page_link.get_attribute("href"))
        print(next_page_link.text)
        driver.execute_script("arguments[0].scrollIntoView(true);", next_page_link)
        time.sleep(1)
        href = next_page_link.get_attribute("href")
        if href and "javascript" not in href:
            driver.get(href)
        else:
            try:
                next_page_link.click()
            except:
                driver.execute_script("arguments[0].click();", next_page_link)
        time.sleep(2)
        page += 1

    except Exception as e:
        print("No more pages or error in pagination:", e)
        break


NameError: name 'webdriver' is not defined

# Load the data collected to Panda Dataset

In [ ]:
data = {"Company_Name" :name, "Email_Address":email, "Phone_Number":phone, "Address":address}
dataset = pd.DataFrame(data)
dataset.to_csv("Datasets\\Motorcycle_Spare_Parts-2.csv")

# Data Cleaning

### Duplicate Removal

In [16]:
print(f"Shape of the dataset before dropping duplicates: {dataset.shape}")
dataset.drop_duplicates()
print(f"Shape of the dataset after dropping duplicates: {dataset.shape}")

Shape of the dataset before dropping duplicates: (1341, 5)
Shape of the dataset after dropping duplicates: (1341, 5)


### Find Null Values

In [17]:
dataset.isnull().sum()

Unnamed: 0         0
Company_Name       0
Email_Address      0
Phone_Number     103
Address          137
dtype: int64

### Drop missing rows (missing rows are saved in another CSV)

In [19]:
# Save it in other csv
dataset[dataset["Phone_Number"].isnull() | dataset["Address"].isnull()].to_csv("Datasets\\Removed_Rows.csv")

# Drop them
print(f"Shape of the dataset after dropping Null containing rows: {dataset.shape}")
dataset = dataset.dropna(subset=["Phone_Number", "Address"])
print(f"Shape of the dataset after dropping Null containing rows: {dataset.shape}")

Shape of the dataset after dropping Null containing rows: (1341, 5)
Shape of the dataset after dropping Null containing rows: (1112, 5)


# Final CSV

In [ ]:
dataset.to_csv("Datasets\\Indonesia.csv")